<a href="https://colab.research.google.com/github/rtajeong/AI_Cluster/blob/main/lab_62_naver_movie_Tfidf_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naver Movie sentiment analysis
- using Tfidf vectorizer

- 감성분석
- 네이버 영화평점 (Naver sentiment movie corpus v.1.0) 데이터(https://github.com/e9t/nsmc)
- 영화 리뷰 20만건이 저장됨. 각 평가 데이터는 0(부정), 1(긍정)으로 label 됨.

한글 자연어 처리
- KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지입니다.
- konlpy 패키지에서 제공하는 Twitter라는 문서 분석 라이브러리 사용 (트위터 분석 뿐 아니라 한글 텍스트 
  처리도 가능)
- colab 사용 권장

In [ ]:
!pip install konlpy

In [ ]:
# 패키지 설치
import konlpy
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn import model_selection, metrics

# 토큰 파서
def twitter_tokenizer(text):
    return Twitter().morphs(text)

In [ ]:
twitter_tokenizer("Welcome to data science world!...한국말도 똑 같아요...")

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['Welcome',
 'to',
 'data',
 'science',
 'world',
 '!...',
 '한국말',
 '도',
 '똑',
 '같아요',
 '...']

In [ ]:
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   152  100   152    0     0   1345      0 --:--:-- --:--:-- --:--:--  1345
100   148    0   148    0     0    360      0 --:--:-- --:--:-- --:--:--     0
100   318  100   318    0     0    476      0 --:--:-- --:--:-- --:--:--   476
100 14.0M  100 14.0M    0     0  4575k      0  0:00:03  0:00:03 --:--:-- 9076k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   151  100   151    0     0   1360      0 --:--:-- --:--:-- --:--:--  1348
100   147    0   147    0     0    379      0 --:--:-- --:--:-- --:--:--   379
100   318  100   318    0     0    511      0 --:--:-- --:--:-- --:--:--   511
100 4827k  100 4827k    0     0  1862k      0  0:00:02  0:00:02 --:--:-- 3503k


In [ ]:
# 데이터 로드
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)


In [ ]:
df_train[:5]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
df_test[:5]

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
df_train['document'].values == df_train['document'].to_numpy()

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
text_train, y_train = df_train['document'].to_numpy(), df_train['label'].to_numpy()
text_test, y_test = df_test['document'].to_numpy(), df_test['label'].to_numpy()

In [ ]:
text_train.shape, y_train.shape, text_test.shape, y_test.shape

((150000,), (150000,), (50000,), (50000,))

In [ ]:
# too much... -> let's take few of them
text_train, y_train = text_train[:2000], y_train[:2000]
text_test, y_test = text_test[:1000], y_test[:1000]

In [ ]:
y_train.shape, y_test.shape

((2000,), (1000,))

In [ ]:
y_train.mean(), y_test.mean()    # check distribution of classes 1 and 0

(0.4945, 0.508)

In [ ]:
cv = TfidfVectorizer(tokenizer=twitter_tokenizer, max_features=3000)
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 3000), (2000,), (1000, 3000), (1000,))

In [ ]:
cv.get_feature_names()[-5:]

['힘들건데', '힘들게', '힘들다', '힘들었네요', '힘들었음']

# Linear Classification

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
result = clf.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.985
Testing :  0.739


# 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
result = lr.fit(X_train,y_train)
feature_names = cv.get_feature_names()
print("Training : ", result.score(X_train, y_train))
print("Testing : ", result.score(X_test, y_test))

Training :  0.916
Testing :  0.771


# MLP

In [ ]:
# use one-hot encoded target
from tensorflow.keras.utils import to_categorical
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)
max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,), activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train_ohe, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train_ohe))
print("Testing : ", model.evaluate(X_test.toarray(), y_test_ohe))

Epoch 1/5
20/20 [==============================] - 3s 4ms/step - loss: 0.6917 - accuracy: 0.5410
Epoch 2/5
20/20 [==============================] - 0s 4ms/step - loss: 0.6694 - accuracy: 0.7450
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.5885 - accuracy: 0.8615
Epoch 4/5
20/20 [==============================] - 0s 4ms/step - loss: 0.4187 - accuracy: 0.8885
Epoch 5/5
63/63 [==============================] - 0s 3ms/step - loss: 0.1596 - accuracy: 0.9595
Training :  [0.15957583487033844, 0.9595000147819519]
32/32 [==============================] - 0s 3ms/step - loss: 0.4986 - accuracy: 0.7540
Testing :  [0.4986066520214081, 0.7540000081062317]


In [ ]:
# use binary target

max_words = X_train.shape[1]
batch_size = 100
nb_epoch = 5

model = Sequential()
model.add(Dense(64, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train.toarray(), y_train, epochs=nb_epoch, 
          batch_size=batch_size) 
print("Training : ", model.evaluate(X_train.toarray(), y_train))
print("Testing : ", model.evaluate(X_test.toarray(), y_test))

Epoch 1/5
20/20 [==============================] - 1s 3ms/step - loss: 0.6910 - accuracy: 0.5560
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6697 - accuracy: 0.7800
Epoch 3/5
20/20 [==============================] - 0s 4ms/step - loss: 0.5963 - accuracy: 0.8635
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.4505 - accuracy: 0.8945
Epoch 5/5
63/63 [==============================] - 1s 3ms/step - loss: 0.2006 - accuracy: 0.9540
Training :  [0.20061413943767548, 0.9539999961853027]
32/32 [==============================] - 0s 3ms/step - loss: 0.4924 - accuracy: 0.7530
Testing :  [0.4923769235610962, 0.753000020980835]


# RNN

In [ ]:
# just for checking
X_train.A[0] == X_train.toarray()[0]

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2000, 3000), (1000, 3000), (2000,), (1000,))

In [ ]:
# RNN 학습을 위한 데이터 재배열
X_train_rnn = X_train.A.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_rnn = X_test.A.reshape((X_test.shape[0], 1, X_test.shape[1]))

print(X_train_rnn.shape, X_test_rnn.shape)

(2000, 1, 3000) (1000, 1, 3000)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(128, 
                    input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                    return_sequences=True))
# return_sequences: return the last output in the output sequence, or the full sequence
# By this, it is possible to access the hidden state output for each input time step.
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])   

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 128)            400512    
_________________________________________________________________
activation_1 (Activation)    (None, 1, 128)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 2s 24ms/step - loss: 0.6887 - accuracy: 0.5465
Epoch 2/5
20/20 [==============================] - 0s 6ms/step - loss: 0.6325 - accuracy: 0.7965
Epoch 3/5
20/20 [==============================] - 0s 6ms/step - loss: 0.4334 - accuracy: 0.8890
Epoch 4/5
20/20 [==============================] - 0s 6ms/step - loss: 0.2157 - accuracy: 0.9240
Epoch 5/5
20/20 [==============================] - 0s 7ms/step - loss: 0.1026 - accuracy: 0.9690


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.765
63/63 [==============================] - 1s 4ms/step - loss: 0.0617 - accuracy: 0.9870
Training :  [0.061724983155727386, 0.9869999885559082]
32/32 [==============================] - 0s 3ms/step - loss: 0.5956 - accuracy: 0.7650
Testing :  [0.5956090688705444, 0.7649999856948853]


# LSTM
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(128, 
               input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
               return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 6s 10ms/step - loss: 0.6931 - accuracy: 0.5375
Epoch 2/5
20/20 [==============================] - 0s 10ms/step - loss: 0.6903 - accuracy: 0.8005
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 0.6703 - accuracy: 0.8725
Epoch 4/5
20/20 [==============================] - 0s 10ms/step - loss: 0.5868 - accuracy: 0.8930
Epoch 5/5
20/20 [==============================] - 0s 10ms/step - loss: 0.4041 - accuracy: 0.9105


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.774
63/63 [==============================] - 1s 5ms/step - loss: 0.2751 - accuracy: 0.9315
Training :  [0.27514004707336426, 0.9315000176429749]
32/32 [==============================] - 0s 5ms/step - loss: 0.4868 - accuracy: 0.7740
Testing :  [0.4867841601371765, 0.7739999890327454]


# GRU

In [ ]:
from keras.layers import GRU
model = Sequential()
model.add(GRU(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(GRU(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rnn, y_train_ohe, batch_size = 100,
          epochs=nb_epoch)

Epoch 1/5
20/20 [==============================] - 4s 10ms/step - loss: 0.6921 - accuracy: 0.5170
Epoch 2/5
20/20 [==============================] - 0s 10ms/step - loss: 0.6716 - accuracy: 0.7150
Epoch 3/5
20/20 [==============================] - 0s 10ms/step - loss: 0.5628 - accuracy: 0.8495
Epoch 4/5
20/20 [==============================] - 0s 11ms/step - loss: 0.3357 - accuracy: 0.9130
Epoch 5/5
20/20 [==============================] - 0s 10ms/step - loss: 0.1801 - accuracy: 0.9390


In [ ]:
y_pred = np.argmax(model.predict(X_test_rnn), axis=1)
print("accuracy score: ", metrics.accuracy_score(y_test, y_pred))
print("Training : ", model.evaluate(X_train_rnn, y_train_ohe))
print("Testing : ", model.evaluate(X_test_rnn, y_test_ohe))

accuracy score:  0.766
63/63 [==============================] - 1s 5ms/step - loss: 0.1123 - accuracy: 0.9670
Training :  [0.11229105293750763, 0.9670000076293945]
32/32 [==============================] - 0s 5ms/step - loss: 0.5520 - accuracy: 0.7660
Testing :  [0.552036702632904, 0.765999972820282]


# Exercise

### 한국어 불용어 처리
- 한국어  불용어 확인은 형태소 분석 라이브러리인 KonLPy 를 이용하면 됨.
- (예) 한국어 품사 중 조사를 추출하는 예
- pos (part-of-speech): 품사 (명사, 동사, ...)

In [ ]:
from konlpy.tag import Twitter

In [ ]:
Twitter().morphs("텍스트 데이터를 이용해서 불용어 사전을 구축하기 위한 간단 예제")

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['텍스트',
 '데이터',
 '를',
 '이용',
 '해서',
 '불',
 '용어',
 '사전',
 '을',
 '구축',
 '하기',
 '위',
 '한',
 '간단',
 '예제']

In [ ]:
Twitter().pos("텍스트 데이터를 이용해서 불용어 사잔을 구축하기 위한 간단 예제")

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('해서', 'Verb'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사잔', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun'),
 ('하기', 'Verb'),
 ('위', 'Noun'),
 ('한', 'Josa'),
 ('간단', 'Noun'),
 ('예제', 'Noun')]

In [ ]:
Twitter().pos("텍스트 데이터를 이용해서 불용어 사전을 구축하기 위한 간단 예제", norm=True)   # norm - 오타 수정 (사잔->사전)

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('해서', 'Verb'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun'),
 ('하기', 'Verb'),
 ('위', 'Noun'),
 ('한', 'Josa'),
 ('간단', 'Noun'),
 ('예제', 'Noun')]

In [ ]:
Twitter().nouns("텍스트 데이터를 이용해서 불용어 사전을 구축하기 위한 간단 예제")

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['텍스트', '데이터', '이용', '불', '용어', '사전', '구축', '위', '간단', '예제']

- norm: 오타수정, stem: 어근 찾기

In [ ]:
from konlpy.tag import Twitter

word_tags = Twitter().pos("텍스트 데이터를 이용해서 불용어 사전을 구축하기 위한 간단 예제", norm=True, stem=True)
print(word_tags)
stop_words = [word[0] for word in word_tags if word[1]=="Josa"]
print (stop_words)

[('텍스트', 'Noun'), ('데이터', 'Noun'), ('를', 'Josa'), ('이용', 'Noun'), ('하다', 'Verb'), ('불', 'Noun'), ('용어', 'Noun'), ('사전', 'Noun'), ('을', 'Josa'), ('구축', 'Noun'), ('하다', 'Verb'), ('위', 'Noun'), ('한', 'Josa'), ('간단', 'Noun'), ('예제', 'Noun')]
['를', '을', '한']


/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


### Pickling

- “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into an object hierarchy.
- Pickling (and unpickling) is alternatively known as “serialization”, “marshalling,” or “flattening”; however, to avoid confusion, the terms “pickling” and “unpickling” are being mostly used.
- Comparison with json
  - There are fundamental differences between the pickle protocols and JSON (JavaScript Object Notation):

  - JSON is a text serialization format (it outputs unicode text, although most of the time it is then encoded to utf-8), while pickle is a binary serialization format;
  - JSON is human-readable, while pickle is not;
  - JSON is interoperable and widely used outside of the Python ecosystem, while pickle is Python-specific;
  - JSON, by default, can only represent a subset of the Python built-in types, and no custom classes; pickle can represent an extremely large number of Python types (many of them automatically, by clever usage of Python’s introspection facilities; complex cases can be tackled by implementing specific object APIs);
  - Unlike pickle, deserializing untrusted JSON does not in itself create an arbitrary code execution vulnerability.

In [ ]:
cv_ = TfidfVectorizer(tokenizer=twitter_tokenizer, max_features=3000)

In [ ]:
import os
import pickle
if not os.path.isfile("X_data.pickle"): 
    print('file does not exists')
    X_data_pre = cv_.fit_transform(text_train)
    pickle.dump(X_data_pre, open("X_data.pickle", "wb"))

file does not exists


/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
# 저장된 tfidf vector 데이터 읽기
with open('X_data.pickle', 'rb') as f:
    X_data_post = pickle.load(f)

In [ ]:
X_data_pre.toarray() == X_data_post.toarray()

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])